In [1]:
from pathlib import Path
from typing import List
from dataclasses import dataclass
import pandas as pd
import tqdm

@dataclass
class SentenceWithName:
    sid: str
    tokens: List[str]
    name_loc: int

def knp_to_sentences(knp: Path) -> List:
    # The sentences are extracted with sentence id(sid), tokens(sent) and location of name tokens (name_loc)
    # Note that the name_loc here is on char level
    sids, sents, name_loc = list(), list(), list()

    with open(knp, 'r') as f:
        sid = None
        tks = list()
        loc = list()
        for line in f:
            if line.startswith("*") or line.startswith("+"):
                # Skip dependencies
                continue
            if line.startswith("# S-ID:"):
                sid = line.split("# S-ID:")[1].split()[0]
                continue
            if line.strip() == "EOS":
                if loc:
                    # The sentence contains names
                    sids.append(sid)
                    sents.append(''.join(tks))
                    name_loc.append(loc)

                sid = None
                tks = list()
                loc = list()
                continue
            token, labels = line.split(' ', maxsplit=1)
            if "人名" in labels:
                # locate the name on character level
                s = len(''.join(tks))
                e = s + len(token)
                # convert [(a, b), (b, c)] to [(a, c)]
                if loc and loc[-1][1] == s:
                    loc[-1][1] = e
                else:
                    loc.append([s, e])
            tks.append(token)
    return sids, sents, name_loc


base_path = Path(".")

files = list(base_path.glob("**/*.knp"))
sids, sents, name_loc = list(), list(), list()
for p in tqdm.tqdm(files):
    s, t, l = knp_to_sentences(p)
    sids += s
    sents += t
    name_loc += l

100%|██████████| 5127/5127 [00:03<00:00, 1384.74it/s]


5127
983


In [2]:
df = pd.DataFrame({
    "id": sids,
    "text": sents,
    "labels": name_loc
})
df.to_csv("sentences.csv", index=False)

[19, 23]


In [3]:
!head sentences.csv

id,text,labels
w201106-0001277619-1,布川事件を描いたドキュメンタリー映画「ショージとタカオ」の試写会に行ってきました。,"[[19, 23], [24, 27]]"
w201106-0001292454-3,東京・霞が関の国交省で、小泉俊明政務官から近藤隆則高梁市長に認定証が手渡された。,"[[12, 16], [21, 25]]"
w201106-0001231705-1,こんにちはソロルファクトリーのサトウナオコです。,"[[15, 21]]"
w201106-0001241209-2,土俵入りの型には雲龍型と不知火型がある。,"[[8, 10], [12, 15]]"
w201106-0001241209-3,それぞれ１０代横綱・雲龍久吉と１１代横綱・不知火光右衛門の型が伝えられたものであるとされている。,"[[10, 14], [21, 28]]"
w201106-0001290480-1,藩命にて武器や軍艦の買い付けに長崎へとやってきた土佐藩参政・後藤象二郎の居宅跡です。,"[[30, 35]]"
w201106-0001290480-2,後に岩崎弥太郎が譲り受けました。,"[[2, 7]]"
w201106-0001243550-1,ジェイデータは、前米副大統領アルバート・ゴア・ジュニア氏が同社の戦略顧問に就任したと発表した。,"[[14, 19], [20, 22], [23, 27]]"
w201106-0001243550-3,また、副大統領時代の経済および情報通信戦略顧問だったグレッグ・サイモン氏もゴア氏の補佐役として実務面を担当する。,"[[26, 30], [31, 35], [37, 39]]"
